# Identity, Immutability, and .NET Polyglot Notebooks

Learning objectives:

- Learn to create documentation in Polyglot Notebooks
- Understand how immutable data structures work in .NET
- Recognize the benefits of immutable data structures
- Differentiate between intrinsic and extrinsic identity
- Convert an API to use intrinsic identity
- Model a system using immutable records
- Project immutable history into current state
- Determine how state changes as history evolves

## Create documentation in Polyglot Notebooks

Install Visual Studio Code from https://code.visualstudio.com/.
Search the extensions marketplace for "Polyglot Notebooks".

Create a new file using the "Polyglot Notebook: create new blank workbook" command.
Choose the *.ipynb file extension.

Create Markdown and Code cells.
Click the language selector in the bottom right corner of the cell to change the language.

Type some code in a code cell.
Press Ctrl+Enter to run the code.
If the last line of code is an expression, the result is displayed below the cell.

## Immutable data structures in .NET

Immutability is the conscious decision to not change data.
It is a design choice that has many benefits.

In .NET, we can create immutable data structures using records.

In [1]:
record Person(String name, DateTime dateOfBirth) {}

var michael = new Person("Michael Perry", new DateTime(1971, 5, 10));

michael

name,Michael Perry
dateOfBirth,1971-05-10 00:00:00Z


In [2]:
int AgeOf(Person person)
{
  var today = DateTime.Today;
  var age = today.Year - person.dateOfBirth.Year;
  if (person.dateOfBirth > today.AddYears(-age)) age--;
  return age;
}

AgeOf(michael)

52

.NET also provides immutable collections.
A useful one is `ImmutableArray<T>`.

In [3]:
using System.Collections.Immutable;

var letters = ImmutableArray.Create('a', 'b', 'c');

letters

[ a, b, c ]

You cannot change an immutable collection.

In [4]:
letters.Add('d');

letters

[ a, b, c ]

You can just create new immutable collections from existing ones.

In [5]:
var moreLetters = letters.Add('d');

moreLetters

[ a, b, c, d ]

## Tic-Tac-Toe Example

One of the best reasons to use immutable data structures is to search a space.
For example, in a game of Tic-Tac-Toe, we can search the space of all possible moves.
I've created a Tic-Tac-Toe API that uses immutable data structures.

In [6]:
#r "TicTacToe\bin\Debug\net7.0\TicTacToe.dll"

using TicTacToe;

In [7]:
var game = Game.Empty
  .Play(4);

game.Html

<!DOCTYPE html>
 
 
 Tic-Tac-Toe Board 
 
 
 
 X

In [8]:
game = game.Play(1);

game.Html

<!DOCTYPE html>
 
 
 Tic-Tac-Toe Board 
 
 
 
 O X

Because we are using immutable data structures, I can produce the next state of the game without destroying the previous state.
This makes it easier to search the space of all possible moves.

In [9]:
game.EmptySquares.Select(square => game.Play(square).Html)

index,value
0,<!DOCTYPE html> Tic-Tac-Toe Board XOX
1,<!DOCTYPE html> Tic-Tac-Toe Board OXX
2,<!DOCTYPE html> Tic-Tac-Toe Board OXX
3,<!DOCTYPE html> Tic-Tac-Toe Board OXX
4,<!DOCTYPE html> Tic-Tac-Toe Board OXX
5,<!DOCTYPE html> Tic-Tac-Toe Board OXX
6,<!DOCTYPE html> Tic-Tac-Toe Board OXX


We can write a function that evaluates a game and predicts who will win.
If the game is an immediate win, then we return the winner.
If there are no more moves, then we return a draw.
Otherwise, we recursively evaluate all possible moves for the next player.

If one of those is a win for the current player, then we assume they will make that winning move.
If there is no win, but there is a draw, then we assume the current player will make that move.
Otherwise, we assume the opponent will win.

In [10]:
Symbol Evaluate(Game game)
{
  var winner = game.Winner;
  if (winner != Symbol.Empty)
    return winner;
  if (!game.EmptySquares.Any())
    return Symbol.Empty;

  var outcomes = game.EmptySquares
    .Select(square => game.Play(square))
    .Select(nextGame => Evaluate(nextGame))
    .ToImmutableArray();
  if (outcomes.Any(outcome => outcome == game.NextPlayer))
    return game.NextPlayer;
  if (outcomes.Any(outcome => outcome == Symbol.Empty))
    return Symbol.Empty;
  return outcomes.First();
}

game.EmptySquares.Select(square => game.Play(square))
  .Select(nextGame => nextGame.HtmlWithOutcome(Evaluate(nextGame)))
  .ToImmutableArray()

index,value
0,<!DOCTYPE html> Tic-Tac-Toe Board XOXX wins
1,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
2,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
3,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
4,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
5,<!DOCTYPE html> Tic-Tac-Toe Board OXXDraw
6,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
